# Group 1

Fabio Ceruti- Fabio Martino - Alex Lucchini - Jacopo Biggiogera

This notebook contains the code that we used to run the CLIP italian model (from the original team including Giuseppe Attanasio). We reverse negeneered this code so as to be able to have it extract blacken out the most salient part of an image.

We then egnerated custom utility functions that would go and smooth out the bounding boxes so as fo rthem to be similar to human generated ones. In addition, we added code that would allow us to save both a msked picture and only teh content of the bouding box

In [1]:
!pip install -q jax
!pip install -U jax[cuda11_cudnn82] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q flax
!pip install -q sentence_transformers
!pip install -q transformers
!wget -q -N https://github.com/huggingface/transformers/raw/master/examples/research_projects/jax-projects/hybrid_clip/modeling_hybrid_clip.py
!wget -q -N https://github.com/huggingface/transformers/raw/master/examples/research_projects/jax-projects/hybrid_clip/configuration_hybrid_clip.py

You should consider upgrading via the '/anaconda/envs/py38_default/bin/python -m pip install --upgrade pip' command.
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
You should consider upgrading via the '/anaconda/envs/py38_default/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/anaconda/envs/py38_default/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/anaconda/envs/py38_default/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/anaconda/envs/py38_default/bin/python -m pip install --upgrade pip' command.


In [2]:
# Importing stuff
import os 
os.environ['TOKENIZERS_PARALLELISM'] = "false"
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

import transformers
from transformers import AutoTokenizer
from jax import numpy as jnp
import torch
from torchvision import datasets, transforms
from torchvision.transforms import CenterCrop, Normalize, Resize, ToTensor
from torchvision.transforms.functional import InterpolationMode
from tqdm.notebook import tqdm
#from modeling_hybrid_clip import FlaxHybridCLIP
from transformers import FlaxVisionTextDualEncoderModel, VisionTextDualEncoderModel

# Utils

In [4]:
######################################ITALIAN################################################
# Setting up the model
from transformers import AutoProcessor, AutoModel

model = FlaxVisionTextDualEncoderModel.from_pretrained("clip-italian/clip-italian")
tokenizer = AutoTokenizer.from_pretrained(
    "dbmdz/bert-base-italian-xxl-uncased", cache_dir=None, use_fast=True
)

image_size = model.config.vision_config.image_size

preprocess = transforms.Compose([
    ToTensor(),
    Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
]) #transforms the image in a tensor and normalises them somehow

In [5]:
# defining necessary functions

def pad_to_square(image, size=224):
    old_size = image.size
    ratio = float(size) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    image = image.resize(new_size, Image.ANTIALIAS)
    new_image = Image.new("RGB", size=(size, size), color=(128, 128, 128))
    new_image.paste(image, ((size - new_size[0]) // 2,
                        (size - new_size[1]) // 2))
    return new_image

#gets embeddings for text
def text_encoder(text):
    inputs = tokenizer([text], max_length=96, truncation=True, padding="max_length", return_tensors="np")
    embedding = model.get_text_features(inputs['input_ids'], inputs['attention_mask'])[0]
    embedding /= jnp.linalg.norm(embedding)
    return embedding

#gets embedding for image
def image_encoder(image):
    image = np.transpose(image, (0, 2, 3, 1))
    features = model.get_image_features(image)
    features /= jnp.linalg.norm(features, keepdims=True)
    return features


def gen_image_batch(path, image_size=224, pixel_size=10):

    n_pixels = image_size // pixel_size + 1 #dividing resized image in to how many squares we will have that we can mask

    image_batch = []
    masks = []

    #saving version of original image with no masks + its grayed out version + its mask (which will be made of all 1's as there is nothing to mask)
    image = Image.open(path).convert("RGB")
    image = pad_to_square(image, size=image_size)
    gray = np.ones_like(image) * 128
    mask = np.ones_like(image)


    image_batch.append(image)
    masks.append(mask)


    for i in range(0, n_pixels): #iterating through each pixel block
        for j in range(i+1, n_pixels): #iterating through all the blocks that come after that
            m = mask.copy() #creating a copy of the original mask with all 1's
            
            #with these two lines they iteratively set all the rows below and above a certain row in the image to 0 (i.e. only the row they want to mask now has 1s)
            m[:min(i*pixel_size, image_size) + 1, :] = 0
            m[min(j*pixel_size, image_size) + 1:, :] = 0
            
            neg_m = 1 - m #with this they invert the content of the matrix so that the only line that was 1s is now the only one with 0 (i.e. they mask a single line)
            image_batch.append(image * m + gray * neg_m) #apply these masks to the image
            masks.append(m)

    #this second for loop does the same thing for columns
    for i in range(0, n_pixels+1):
        for j in range(i+1, n_pixels+1):
            m = mask.copy()
            m[:, :min(i*pixel_size + 1, image_size)] = 0
            m[:, min(j*pixel_size + 1, image_size):] = 0
            neg_m = 1 - m
            image_batch.append(image * m + gray * neg_m)
            masks.append(m)

    return image_batch, masks #returns two lists.
    #image_batrch: contains a list with all the pictures (in array form) where in each either one row or one cloumn has been masked
    #masks: contain matrices where all lines are 0s excpet the row/column that one wants to mask (which has 1s instead)


def get_heatmap(image_path, text, pixel_size=10, iterations=3):
    text_embedding = text_encoder(text) #get vectors for the text by applying the model
    images, masks = gen_image_batch(image_path, pixel_size=pixel_size) #generates the two lists described above

    images = np.stack([preprocess(image) for image in images], axis=0)
    image_embeddings = jnp.asarray(image_encoder(images)) #encode each of the masked images using the pretrained model and save them in one big array
    
    sims = []
    scores = []
    mask_val = jnp.zeros_like(masks[0]) #this will be the final mask that is applied to the image and will be iteratively updated withe the masks we check in the loop below

    for e, m in zip(image_embeddings, masks):
        sim = jnp.matmul(e, text_embedding.T) #multiply the text and the specific mask vector representation to get a number representing the weight of the vector representation that mask once multiplied to the text
        sims.append(sim) #store this number in a list
        if len(sims) > 1:
            scores.append(sim * m) #this gives you a matrix where everything is 0 expect in the row that is masked, where now each number is not 1 but a weight that is the sim value 
            mask_val += 1-m # update the 'master mask' by adding other rows and columns to mask by adding inverses of m matrices

    #sims[0] contains the score for the whole picture without masks, by subtracting this score value we essentially remove all pixels that
    #have scores that are too low to begin with when compared to the whole picture
    score = jnp.mean(jnp.clip(jnp.array(scores) - sims[0], 0, jnp.inf), axis=0)
    
    #this for loop essentially allows you to choose how specific you want to be in knowing what the network pays attention to.
    #by increasing the iterations you will subtract many times the mean of the score matrix to each element in score, thus only allowing 
    #pixels that have a very high score to 'survive' and be displayed.
    for i in range(iterations):
        score = jnp.clip(score - jnp.mean(score), 0, jnp.inf) #here you subtract the mean of score to eachh element of core as descirbed above

    score = (score - jnp.min(score)) / (jnp.max(score) - jnp.min(score)) #normalising each value to be between 0 and 1
    
    #reversed = np.asarray(score)
    #reversed = reversed[reversed >0] = 1
    #reversed = 1-reversed
    ar = []
    for r in np.asarray(score):
      row = []
      for item in r:
        row.append(item)
      ar.append(row)
    new = np.array(ar)
    new[new>0]=1
    rev = 1-new
    return np.asarray(score), rev


def display_heatmap(image, text, heatmap, rev):
    fig = plt.figure(figsize=(15, 30), facecolor='white')
    ax1 = fig.add_subplot(131)
    ax2 = fig.add_subplot(132)
    ax3 = fig.add_subplot(133)
    ax4 = fig.add_subplot(133)

    ax1.imshow(image)
    ax2.imshow(heatmap)
    ax3.imshow(np.asarray(image) / 255. * heatmap)
    ax4.imshow(np.asarray(image) / 255. * rev)

    ax1.axis('off')
    ax2.axis('off')
    ax3.axis('off')
    ax4.axis('off')

    ax1.title.set_text('input image')
    ax2.title.set_text(text)

    plt.tight_layout()

    plt.show()

In [6]:
def masked_img_array(image, rev):
    '''This function returns an array of a passed image with the target padded'''
    return np.asarray(image) / 255. * rev

def de_padder(image, image_old):
    original_size = image_old.size 
  
    x = np.array(image)*255
    rows_to_drop = []
    columns_to_drop = []

    #iterate through rows to find ones that are grey
    for ind, row in enumerate(x[:, :, 0]):
        isgray = []
        for i in row:
            if (i == 128) | (i == 0):
                isgray.append(True)
            else:
                isgray.append(False)
        isgray = np.array(isgray)
        if isgray.all():#| len(isgray[isgray==True])>=len(isgray)/2:
            rows_to_drop.append(ind)
  

    clean_ofrows = np.delete(x, rows_to_drop, axis = 0)
    #iterate through columns
    cols_to_drop = []
    for ind in range(clean_ofrows.shape[1]):
        col = clean_ofrows[:, ind, 0]
        zeroes = col[col==0]
        onete = col[col==128]
        if len(zeroes)+len(onete) == len(col):
            cols_to_drop.append(ind)
        #isgray = []
        #for i in clean_ofrows[:, :, 0]:
         #   if (i[ind] == 128) | (i[ind] == 0):
          #      isgray.append(True)
           # else:
            #    isgray.append(False)
        #isgray = np.array([i[ind]==128 for i in clean_ofrows])
        #isgray = np.array(isgray)

        #if isgray.all(): #| len(isgray[isgray==True])>=len(isgray)/2:
            #cols_to_drop.append(ind)
            
    clean = np.delete(clean_ofrows, cols_to_drop, axis = 1)
    clean_im = Image.fromarray(np.uint8(clean))
    clean_im = clean_im.resize(original_size, Image.ANTIALIAS)


    #replace ones that are zero in masked image in the high res picture so as to keep it all high res
    zero_ind = np.where(np.array(clean_im) == 0)
    old = np.array(image_old)
    bbox_content = np.array(image_old)

    #finding all pixels that were balck originally 
    black_original = np.where(old == 0)
    
    widths = list(sorted(set(zero_ind[0])-set(black_original[0])))
    start_w = min(widths[2:])
    end_w = max(widths[:-5])

    #finding columns within the row subset
    black_original_cols= np.where(old[start_w:end_w, :, :] == 0)
    zero_ind_cols= np.where(np.array(clean_im)[start_w:end_w, :, :] == 0)
 
    heights = list(sorted(set(zero_ind_cols[1])-set(black_original_cols[1])))

    #start_w = min(widths[2:])
    #end_w = max(widths[:-5])
    start_h = min(heights[2:])
    end_h = max(heights[:-2])
    old[start_w:end_w, start_h:end_h, 0] =0
    old[start_w:end_w, start_h:end_h, 1] =0
    old[start_w:end_w, start_h:end_h, 2] =0

    box_1 = bbox_content[start_w:end_w, start_h:end_h, 0]
    box_2 = bbox_content[start_w:end_w, start_h:end_h, 1] 
    box_3 = bbox_content[start_w:end_w, start_h:end_h, 2]
    box = np.zeros((box_1.shape[0], box_1.shape[1], 3))
    box[:,:,0] = box_1
    box[:,:,1] = box_2
    box[:,:,2] = box_3

    highres_masked = Image.fromarray(np.uint8(old))
    highres_bboxcontent = Image.fromarray(np.uint8(box))
    return highres_masked, highres_bboxcontent
  



## Checking Code works before iterating though all pictures

In [8]:
#open image
image = Image.open('Animals/Sciurus carolinensis/3709255098963210ccd3dcdac941c82f.jpg').convert("RGB")

#pad the image
image2 = pad_to_square(image, size=224)

heatmap, rev= get_heatmap('Animals/Sciurus carolinensis/3709255098963210ccd3dcdac941c82f.jpg', query, pixel_size=5, iterations=15)
masked_img = masked_img_array(image2, rev)
clean_mask, clean_bbox = de_padder(masked_img, image)

In [10]:
#Mask
Image.fromarray(np.uint8(masked_img))

In [11]:
#Heatmap
Image.fromarray(np.uint8(np.asarray(image2) / 255. * heatmap))

# Running the model

In [ ]:
#Making base directories
os.mkdir('test_masked')
os.mkdir('test_bbox')

In [131]:
#Creating directories for each species
for spec in os.listdir('Animals'):
    if spec!= '.DS_Store':
        os.mkdir('test_masked/'+spec)
        os.mkdir('test_bbox/'+spec)

In [1]:
#importing images for which to generate artificial bboxes
import pandas as pd
from tqdm import tqdm
test_pics = pd.read_csv('old_test_data.csv')
test_pics.head()

,pic_name,bbox,train,path,label,species_y
0,c3075d381e62d4fcee781af1c94be073.jpg,NaN,0.0,Animals/Sciurus carolinensis/c3075d381e62d4fce...,9,Sciurus carolinensis
1,219d19b874f616140a33d51d95f5610b.jpg,NaN,0.0,Animals/Sciurus carolinensis/219d19b874f616140...,9,Sciurus carolinensis
2,1b0e269e5e6499e51fb7c450306c9ea0.jpg,NaN,0.0,Animals/Sciurus carolinensis/1b0e269e5e6499e51...,9,Sciurus carolinensis
3,feb80a89a67b30a32d544857b205ce30.jpg,NaN,0.0,Animals/Sciurus carolinensis/feb80a89a67b30a32...,9,Sciurus carolinensis
4,4eef913a2936960c4b009772439a3588.jpg,NaN,0.0,Animals/Sciurus carolinensis/4eef913a2936960c4...,9,Sciurus carolinensis


In [8]:
query_dictionary = {
'Anolis carolinensis': 'una lucertola', 
'Canis latrans':'un coyote',
'Chelydra serpentina':'una tartaruga', 
'Harmonia axyridis':'una coccinella',
'Junonia coenia':'una farfalla', 
'Pachydiplax longipennis':'una libellula', 
'Pandion haliaetus':'un falco', 
'Picoides pubescens':'un picchio',
'Procyon lotor':'un procione', 
'Sciurus carolinensis':'uno scoiattolo',
'Spinus tristis':'un uccello', 
'Trachemys scripta elegans':'una tartaruga'   
}

In [9]:
wrong = []
for pic_path in tqdm(test_pics['path'][3350:]):
    items = pic_path.split('/')
    species = items[1]
    name = items[-1]
    
    #build query
    query = query_dictionary[species]
    #read image
    image = Image.open(pic_path).convert("RGB")
    #pad the image
    image2 = pad_to_square(image, size=224)
    try:
        #generate saliency mask
        heatmap, rev= get_heatmap(pic_path, query, pixel_size=5, iterations=15)
    
        #maks the image
        masked_img = masked_img_array(image2, rev)
        #generate masked image and only bbox content
        clean_mask, clean_bbox = de_padder(masked_img, image)
    
        #save
        clean_mask.save('test_masked/'+species+'/'+name)
        clean_bbox.save('test_bbox/'+species+'/'+name)
    except ValueError:
        wrong.append(pic_path)
        
    

100%|██████████| 783/783 [3:22:26<00:00, 15.51s/it]  


## Saving data

In [11]:
# Saving corrupted images
wrong_pd = pd.DataFrame({'with_mistake':wrong})
wrong_pd.to_csv('wrong_tot.csv', index = False)

In [10]:
import os
#Saving paths to all test images that are now masked
paths = []
pic_name = []
for i in os.listdir('test_masked'):
    for p in os.listdir('test_masked/'+i):
        paths.append('test_masked/'+i+'/'+p)
        pic_name.append(p)
  

In [11]:
#Saving paths to all test images for which we extracted a bounding box
paths_bbox = []
pic_name_bbox = []
for i in os.listdir('test_bbox'):
    for p in os.listdir('test_bbox/'+i):
        paths_bbox.append('test_bbox/'+i+'/'+p)
        pic_name_bbox.append(p)

In [13]:
#Saving as a csv
photos_test_masked = pd.DataFrame({'path':paths, 'pic_name':pic_name})
photos_test_masked.to_csv('photos_test_masked.csv', index = False)
photos_test_bbox = pd.DataFrame({'path':paths_bbox, 'pic_name':pic_name_bbox})
photos_test_bbox.to_csv('photos_test_bbox.csv', index = False)

## Moving pictures to master masked and bbcontent directories

In [22]:
import pandas as pd
import os
import shutil
from tqdm import tqdm
p_bbox = pd.read_csv('photos_test_bbox.csv')
p_masked = pd.read_csv('photos_test_masked.csv')

In [17]:
p_masked.head()

,path,pic_name
0,test_masked/Sciurus carolinensis/c3075d381e62d...,c3075d381e62d4fcee781af1c94be073.jpg
1,test_masked/Sciurus carolinensis/219d19b874f61...,219d19b874f616140a33d51d95f5610b.jpg
2,test_masked/Sciurus carolinensis/1b0e269e5e649...,1b0e269e5e6499e51fb7c450306c9ea0.jpg
3,test_masked/Sciurus carolinensis/feb80a89a67b3...,feb80a89a67b30a32d544857b205ce30.jpg
4,test_masked/Sciurus carolinensis/4eef913a29369...,4eef913a2936960c4b009772439a3588.jpg


In [10]:
paths_bbox_pre = []
pic_name_bbox_pre = []
for i in os.listdir('Animals_bbcontent'):
    for p in os.listdir('Animals_bbcontent/'+i):
        paths_bbox_pre.append('Animals_bbcontent/'+i+'/'+p)
        pic_name_bbox_pre.append(p)

In [12]:
paths_masks_pre = []
pic_name_masks_pre = []
for i in os.listdir('Animals_masked'):
    for p in os.listdir('Animals_masked/'+i):
        paths_masks_pre.append('Animals_masked/'+i+'/'+p)
        pic_name_masks_pre.append(p)

In [15]:
print(f'masks: before:{len(paths_masks_pre)}, should have after: {len(paths_masks_pre)+ 3793}')
print(f'bboxes: before:{len(paths_bbox_pre)}, should have after: {len(paths_bbox_pre)+ 3793}')

masks: before:13302, should have after: 17095
bboxes: before:14942, should have after: 18735


In [24]:
#Moving bboxes
dest_folder_bboxes = 'Animals_bbcontent/'
for picb in tqdm(p_bbox['path']):
    if picb.split('/')[-1] != '.ipynb_checkpoints':
        destination_b = dest_folder_bboxes+picb[10:]
        shutil.copy(picb, destination_b)

#Moving masks
dest_folder_masks = 'Animals_masked/'
for picm in tqdm(p_masked['path']):
    if picm.split('/')[-1] != '.ipynb_checkpoints':
        destination_m = dest_folder_masks+picm[12:]
        shutil.copy(picm, destination_m)

100%|██████████| 3793/3793 [00:02<00:00, 1421.49it/s]


In [25]:
paths_bbox_post= []
pic_name_bbox_post = []
for i in os.listdir('Animals_bbcontent'):
    for p in os.listdir('Animals_bbcontent/'+i):
        paths_bbox_post.append('Animals_bbcontent/'+i+'/'+p)
        pic_name_bbox_post.append(p)

paths_masks_post = []
pic_name_masks_post = []
for i in os.listdir('Animals_masked'):
    for p in os.listdir('Animals_masked/'+i):
        paths_masks_post.append('Animals_masked/'+i+'/'+p)
        pic_name_masks_post.append(p)

In [26]:
print(f'masks: before:{len(paths_masks_post)}, should have after: {len(paths_masks_pre)+ 3793}')
print(f'bboxes: before:{len(paths_bbox_post)}, should have after: {len(paths_bbox_pre)+ 3793}')

masks: before:17092, should have after: 17095
bboxes: before:18732, should have after: 18735


All files have been moved correctly, there are only 3 files that are not there bit this is correct since they are simply '.ipynb_checkpoints' which we filtered out.